In [31]:
import pandas as pd
from haversine import haversine as hvs, Unit
import numpy as np
from collections import Counter
import math
from datetime import datetime, timedelta

# Analisis de Datos

### Busstop ida

In [6]:
# leer dataset busstop
df_stops_ida = pd.read_csv('../data/busstops_ida.csv')

# diseño del df: creacion de latitude_longitude
df_stops_ida['latitude'] = df_stops_ida.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
df_stops_ida.drop(['longitude', 'number'], inplace=True, axis=1)
df_stops = df_stops_ida.rename(columns={'latitude':'latitude_longitude'}, inplace=False)

df_stops['id'] = df_stops['id'].values.astype(str)

# df_stops.info()
df_stops

,id,name,navigation,latitude_longitude,radio
0,0,inicio,159,"(-13.5549784, -71.856025)",30
1,36,primer stop,304,"(-13.5548123, -71.8573001)",18
2,37,segundo stop,281,"(-13.5532944, -71.8619442)",13
3,38,Ccollana,283,"(-13.552296, -71.867299)",9
4,39,Puente Huaccoto,283,"(-13.551592, -71.870211)",10
...,...,...,...,...,...
79,114,Segundo,221,"(-13.547528, -71.985248)",10
80,115,Reservorio,225,"(-13.54966, -71.988003)",10
81,116,Cuarto,230,"(-13.550408, -71.98888)",10
82,117,Quinto,203,"(-13.550841, -71.989448)",10


### Crear links

In [7]:
def generateLinks(row):
    row['end_stop'] = df_linkref.loc[row.name+1, 'id']
    row['name_end_stop'] = df_linkref.loc[row.name+1, 'name']
    row['linkref'] = str(row['id']) + ':' + str(df_linkref.loc[row.name+1, 'id'])
    return row

df_linkref = df_stops.copy()
df_linkref = df_linkref.head(df_linkref.shape[0]-1).apply(generateLinks, axis=1)

df_linkref = df_linkref.drop(df_linkref.columns[2:5], axis=1)
df_linkref = df_linkref.rename(columns={'id':'init_stop'})
df_linkref['end_stop'] = df_linkref['end_stop'].values.astype(str)
df_linkref

,init_stop,name,end_stop,name_end_stop,linkref
0,0,inicio,36,primer stop,0:36
1,36,primer stop,37,segundo stop,36:37
2,37,segundo stop,38,Ccollana,37:38
3,38,Ccollana,39,Puente Huaccoto,38:39
4,39,Puente Huaccoto,40,Kayra,39:40
...,...,...,...,...,...
78,113,Primero,114,Segundo,113:114
79,114,Segundo,115,Reservorio,114:115
80,115,Reservorio,116,Cuarto,115:116
81,116,Cuarto,117,Quinto,116:117


### Leer dataset

In [35]:
# leer dataset monitoring
data = pd.read_csv('../data/_Process_monitoring__202303251137.csv')
# data = pd.read_csv('../data/_Process_monitoring__pruebas.csv')

# diseño del df: creacion data_time y latitude_longitude
data['date_time'] = data['date'] + ' ' + data['time']
data['latitude'] = data.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
data.drop(['longitude'], inplace=True, axis=1)
data = data.rename(columns={'latitude':'latitude_longitude', 'vehicle_id_id':'vehicle_id'}, inplace=False)
data = data.drop_duplicates(subset=['latitude_longitude']) # (65155, 7) (64702, 7)
data = data.sort_values(by='date_time')

# convertir de object a datatime
data['date_time'] = pd.to_datetime(data['date_time'], format='%Y-%m-%d %H:%M:%S')
print(data.shape)
# data['id'] = data.index
data
# a = data.loc[4824, 'date_time']
# b = data.loc[4978, 'date_time']
# print(a)
# print(b)
# (a + abs(a-b)/2).round('1s')


(1671363, 9)


,id,date,time,lap,latitude_longitude,velocity,navigation,vehicle_id,date_time
29756,13172252,2023-03-01,00:01:34,5,"(-13.5513563, -71.9902231)",1,140,SJ40,2023-03-01 00:01:34
29311,13172234,2023-03-01,00:01:49,0,"(-13.5506013, -71.8786173)",2,185,SJ10,2023-03-01 00:01:49
29780,13172253,2023-03-01,00:03:25,0,"(-13.5506231, -71.8786667)",3,206,SJ10,2023-03-01 00:03:25
31234,13172309,2023-03-01,00:05:13,5,"(-13.5383472, -71.9807985)",0,0,SJ08,2023-03-01 00:05:13
29831,13172255,2023-03-01,00:05:30,0,"(-13.55065, -71.8785454)",3,113,SJ10,2023-03-01 00:05:30
...,...,...,...,...,...,...,...,...,...
1585989,17270532,2023-03-25,11:37:16,2,"(-13.5504177, -71.9870624)",31,230,SJ03,2023-03-25 11:37:16
1585942,17270531,2023-03-25,11:37:16,3,"(-13.550582, -71.8741605)",16,285,SJ20,2023-03-25 11:37:16
1586202,17270537,2023-03-25,11:37:17,3,"(-13.5217018, -71.9646513)",19,289,SJ32,2023-03-25 11:37:17
1586071,17270534,2023-03-25,11:37:17,2,"(-13.5405389, -71.9812735)",38,13,SJ04,2023-03-25 11:37:17


### Ver vehiculos

In [36]:
# cantidad de datos: laps y vehicles
vehicles = sorted(data['vehicle_id'].unique())

# datos por cada vehiculo
veh_rows = {}
for veh in vehicles: 
    veh_rows[veh] = len(data[data['vehicle_id'] == veh])

print('# Max laps: ', len(data['lap'].unique()), '->',data['lap'].unique())
print('# Max de datos: ', max(veh_rows.values()))
print('# Min de datos: ', min(veh_rows.values()))
print(veh_rows.items())

# Max laps:  9 -> [5 0 1 2 3 4 6 7 8]
# Max de datos:  52906
# Min de datos:  22395
dict_items([('SJ01', 35181), ('SJ02', 29160), ('SJ03', 42058), ('SJ04', 50224), ('SJ05', 40731), ('SJ06', 39633), ('SJ07', 36598), ('SJ08', 24087), ('SJ09', 37800), ('SJ10', 38377), ('SJ11', 40067), ('SJ12', 41045), ('SJ13', 37694), ('SJ14', 43592), ('SJ15', 46352), ('SJ16', 32610), ('SJ17', 41708), ('SJ18', 39159), ('SJ19', 38227), ('SJ20', 50677), ('SJ21', 36242), ('SJ22', 39382), ('SJ23', 23911), ('SJ24', 47823), ('SJ25', 49012), ('SJ26', 43398), ('SJ27', 48524), ('SJ28', 46360), ('SJ29', 25667), ('SJ30', 33336), ('SJ31', 48083), ('SJ32', 22395), ('SJ33', 41801), ('SJ34', 52906), ('SJ35', 31348), ('SJ36', 27368), ('SJ37', 39065), ('SJ38', 27672), ('SJ39', 39880), ('SJ40', 46163), ('SJ41', 36817), ('SJ42', 43926), ('SJ43', 35304)])


        Funciones

In [24]:
# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if ( itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    nav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (nav<=45 and dis<=stop['radio'])

# FUNCION: VERIFICAR SI EL PUNTO PASO POR UN STOP
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    return row

# FUNCION: calcula el punto medio de tiempo
def midtime(row, df):
    if row['stop'] == df.loc[row.name+1, 'stop']:
        a = df.loc[row.name, 'date_time']
        b = df.loc[row.name+1, 'date_time']
        row['date_time'] = (a - abs(a-b)/2).round('1s') if a<b else (a + abs(a-b)/2).round('1s')
    return row 

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'date_time'], df.loc[row['end_stop'], 'date_time']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']
        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']

    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row

# Calcular Time Travel

In [37]:
# variables de registro
df_his = pd.DataFrame(columns=['date', 'vehicle', 'lap', 'ida_Points' , 'stopsPoints', 'Percent_stopPoints', 'links', 'missingStops','list_missingStops'])
df_sp = pd.DataFrame(columns=['link-' + str(i) for i in range(df_linkref.shape[0])])
list_samples = []

# Procesado de datos
for fecha in data['date'].unique():
    data_f = data[data['date'] == fecha]
    print(fecha)
    for veh in vehicles:
        df_veh = data_f[data_f['vehicle_id'] == veh].reset_index(drop=True).copy()
        laps = sorted(df_veh['lap'].unique()[1:-1])

        for lap in laps:
            # filtrar los puntos de ida
            df_veh_lap = idaPoints(df_veh[df_veh['lap'] == lap].reset_index(drop=True))

            # verificar puntos por stop
            df_veh_lap_stop = df_veh_lap.apply(lambda row: searchStopsPoints(row, df_stops), axis=1)
            df_veh_lap_stop = df_veh_lap_stop[df_veh_lap_stop['stop'].notna()].reset_index()

            # filtrar un unico punto por stop
            df_veh_lap_stop.sort_values(by=['stop', 'dis'], ascending=True, inplace=True, ignore_index=True, key=np.int64)
            # df_veh_lap_stop = df_veh_lap_stop.head(df_veh_lap_stop.shape[0]-1).apply(lambda row: midtime(row, df_veh_lap_stop), axis=1)
            df_veh_lap_stop.drop_duplicates(subset=['stop'], keep='first', inplace=True, ignore_index=True)

            # filtrar los stop sin point 
            missing_stops = list(set(df_stops['id']) - set(df_veh_lap_stop['stop'].unique()))

            # print(df_veh_lap_stop, df_veh_lap_stop.shape)

            if len(missing_stops) < 24:
                # crear sample
                df_sample = df_linkref.copy()
                df_veh_lap_stop.set_index('stop', inplace=True)
                df_sample = df_sample.apply(lambda row: time_travel(row, df_veh_lap_stop), axis=1)
                list_samples.append(df_sample)

                # registrar metricas
                df_sp.loc[df_sp.shape[0]] = list(df_sample['time_travel'])
                idaP, stopP, links, nNaN = df_veh_lap.shape[0], df_veh_lap_stop.shape[0], df_sample.shape[0], df_sample['time_travel'].isna().sum()
                df_his.loc[df_his.shape[0]] = [fecha, veh, lap, idaP, stopP, stopP/idaP, links/83, len(missing_stops), missing_stops]
                

2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-11
2023-03-12
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
2023-03-19
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-25


In [38]:
csv_temp = pd.concat(list_samples)
# csv_temp.to_csv('samples_with_nan.csv')
csv_temp.to_csv('../data/pcampo.csv')

In [28]:
csv_temp

,init_stop,name,end_stop,name_end_stop,linkref,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,inicio,36,primer stop,0:36,2023-07-06 08:56:58,2023-07-06 08:58:25,87.0,8798.0,11645.0,SJ01,2.0,2023-07-06
1,36,primer stop,37,segundo stop,36:37,2023-07-06 08:58:25,2023-07-06 08:59:25,60.0,11645.0,13599.0,SJ01,2.0,2023-07-06
2,37,segundo stop,38,Ccollana,37:38,2023-07-06 08:59:25,2023-07-06 09:00:29,64.0,13599.0,15682.0,SJ01,2.0,2023-07-06
3,38,Ccollana,39,Puente Huaccoto,38:39,2023-07-06 09:00:29,2023-07-06 09:01:15,46.0,15682.0,17180.0,SJ01,2.0,2023-07-06
4,39,Puente Huaccoto,40,Kayra,39:40,2023-07-06 09:01:15,2023-07-06 09:02:05,50.0,17180.0,18858.0,SJ01,2.0,2023-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,113,Primero,114,Segundo,113:114,2023-07-06 16:13:01,2023-07-06 16:13:13,12.0,34996.0,35485.0,SJ43,5.0,2023-07-06
79,114,Segundo,115,Reservorio,114:115,2023-07-06 16:13:13,2023-07-06 16:13:49,36.0,35485.0,17660.0,SJ43,5.0,2023-07-06
80,115,Reservorio,116,Cuarto,115:116,2023-07-06 16:13:49,2023-07-06 16:14:01,12.0,17660.0,18089.0,SJ43,5.0,2023-07-06
81,116,Cuarto,117,Quinto,116:117,2023-07-06 16:14:01,2023-07-06 16:14:13,12.0,18089.0,18587.0,SJ43,5.0,2023-07-06
